# PaddleOCR 다중 이미지(여러 장) 파이프라인 테스트

- 입력 소스/디렉터리 규칙은 `notebooks/ocr/paddleocr_test.ipynb`와 동일하게 유지합니다.
- `img_paths = [...]` 리스트로 여러 장을 처리합니다.
- 각 이미지마다 헤더 + 최종 JSON을 출력하고, 마지막에 `LabReportExtractor.merge_extractions`로 스마트 병합 수행.

> 형님 메모: PyCharm에서 노트북 UI로 열려야 정상입니다.


In [10]:
# 노트북 경로 부트스트랩 (원본 노트북과 동일 패턴)
import sys
from pathlib import Path

cwd = Path.cwd()
project_root = None
p = cwd

for _ in range(6):
    if (p / 'src').exists() and (p / 'pyproject.toml').exists():
        project_root = p
        break
    p = p.parent

if project_root and str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f'✅ 프로젝트 루트: {project_root}')
print(f'✅ sys.path[0]: {sys.path[0]}')


✅ 프로젝트 루트: /home/aidan/projects/meow-chat
✅ sys.path[0]: C:\Program Files\JetBrains\PyCharm 2025.3.1\plugins\python-ce\helpers\jupyter_debug


In [11]:
# 모듈 임포트 (원본 노트북과 동일 패턴)
import os

# 환경변수 설정 (반드시 settings import 전에 실행!)
os.environ['OCR_PROVIDER'] = 'paddle'
os.environ['OCR_USE_GPU'] = 'false'

from src.services.ocr.paddle_ocr import MyPaddleOCR
from src.services.preprocessing.image_preprocessor import ImagePreprocessor
from src.services.lab_extraction.line_preprocessor import LinePreprocessor
from src.services.lab_extraction.lab_table_extractor import LabTableExtractor
from src.services.lab_extraction.lab_report_extractor import LabReportExtractor
from src.settings import settings

print('✅ 모듈 임포트 완료')
print(f'   OCR Provider: {settings.ocr_provider}')
print(f'   OCR Use GPU: {settings.ocr_use_gpu}')


✅ 모듈 임포트 완료
   OCR Provider: paddle
   OCR Use GPU: False


In [14]:
# 인스턴스 생성
ocr_service = MyPaddleOCR(use_gpu=False, lang='korean')
image_preprocessor = ImagePreprocessor()
line_preprocessor = LinePreprocessor()
lab_table_extractor = LabTableExtractor()
# LabReportExtractor는 의존성을 직접 주입
lab_report_extractor = LabReportExtractor(
    line_preproc=line_preprocessor,
    extractor=lab_table_extractor
)

print('ocr_service=', type(ocr_service).__name__)
print('image_preprocessor=', type(image_preprocessor).__name__)
print('line_preprocessor=', type(line_preprocessor).__name__)
print('lab_table_extractor=', type(lab_table_extractor).__name__)
print('lab_report_extractor=', type(lab_report_extractor).__name__)


ocr_service= MyPaddleOCR
image_preprocessor= ImagePreprocessor
line_preprocessor= LinePreprocessor
lab_table_extractor= LabTableExtractor
lab_report_extractor= LabReportExtractor


In [15]:
# 입력: 원본 노트북과 동일하게 assets/images 아래에서 선택
from pathlib import Path

base_dir = project_root / 'notebooks' / 'ocr' / 'assets' / 'images'

# 예시(형님이 수정해서 사용)
img_paths = [
    base_dir / '20241221_0.jpg',
    base_dir / '20241221_1.jpg',
]

# 기본값: img_paths가 비어있을 때만 자동 선택
if not img_paths:
    candidates = sorted([p for p in base_dir.glob('*') if p.suffix.lower() in {'.jpg', '.jpeg', '.png', '.webp'}])
    img_paths = candidates[:2]

print('📂 base_dir=', base_dir)
print('🖼️ img_paths=')
for p in img_paths:
    print(' -', p.name)

if not img_paths:
    raise RuntimeError('img_paths가 비어있습니다. assets/images에 샘플이 있는지 확인하세요.')


📂 base_dir= /home/aidan/projects/meow-chat/notebooks/ocr/assets/images
🖼️ img_paths=
 - 20241221_0.jpg
 - 20241221_1.jpg


In [16]:
# 유틸: 문서 결과 생성/검증
from typing import Any, Dict
import json
import time

DocumentResult = Dict[str, Any]

def init_doc_result() -> DocumentResult:
    return {
        'hospital_name': None,
        'client_name': None,
        'patient_name': None,
        'inspection_date': None,
        'tests': [],
    }

def validate_doc_result(doc: DocumentResult) -> tuple[bool, str]:
    if not isinstance(doc, dict):
        return False, 'doc_result가 dict가 아닙니다.'
    tests = doc.get('tests', [])
    if not tests:
        return False, 'tests가 비어있습니다.'
    return True, f'tests={len(tests)}'

def print_header(idx: int, path: Path, elapsed_s: float, raw_lines: int, tests: int):
    print('\n' + '=' * 80)
    print(f'🧾 [{idx + 1}/{len(img_paths)}] {path.name}')
    print(f'⏱️ elapsed: {elapsed_s:.2f}s | raw_lines: {raw_lines} | tests: {tests}')
    print('=' * 80)


In [17]:
# 다중 이미지 처리 루프
import io
from PIL import Image as PILImage

all_results: list[dict] = []

for i, img_path in enumerate(img_paths):
    t0 = time.time()

    raw_bytes = Path(img_path).read_bytes()
    pre_bytes = image_preprocessor.process_bytes(raw_bytes, debug=False)

    pil_img = PILImage.open(io.BytesIO(pre_bytes))
    ocr_result = ocr_service.extract_text(pil_img)

    raw_lines: list[str] = []
    if ocr_result and getattr(ocr_result, 'data', None) and getattr(ocr_result.data, 'items', None):
        for item in ocr_result.data.items:
            raw_lines.extend(getattr(item, 'rec_texts', []) or [])
    raw_text = '\n'.join(raw_lines)

    doc_result = init_doc_result()
    try:
        if ocr_result and ocr_result.data and ocr_result.data.items:
            page = ocr_result.data.items[0]
            lined_data = line_preprocessor.extract_and_group_lines(page)
            extracted, _intermediates = lab_table_extractor.extract_from_lines(lined_data, return_intermediates=True)
            if extracted:
                doc_result.update(extracted)
    except Exception as e:
        doc_result['__error__'] = str(e)

    elapsed = time.time() - t0
    is_valid, reason = validate_doc_result(doc_result)

    print_header(i, Path(img_path), elapsed, raw_lines=len(raw_lines), tests=len(doc_result.get('tests', []) or []))
    print(f'valid={is_valid} ({reason})')
    print(json.dumps(doc_result, ensure_ascii=False, indent=2))

    all_results.append({
        'image': Path(img_path).name,
        'elapsed_s': elapsed,
        'raw_text': raw_text,
        'doc_result': doc_result,
        'valid': is_valid,
        'valid_reason': reason,
    })


Creating model: ('PP-LCNet_x1_0_doc_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/aidan/.paddlex/official_models/PP-LCNet_x1_0_doc_ori`.
Creating model: ('UVDoc', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/aidan/.paddlex/official_models/UVDoc`.
Creating model: ('PP-LCNet_x1_0_textline_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/aidan/.paddlex/official_models/PP-LCNet_x1_0_textline_ori`.
Creating model: ('PP-OCRv5_server_det', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/aidan/.paddlex/official_models/PP-OCRv5_server_det`.
Creating model: ('korean_PP-OCRv5_mobile_rec', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/aidan/.paddlex/official_mo

[DEBUG] tokens_with_geometry: count=155
[
  {
    "text": "SAN",
    "confidence": 0.9959490895271301,
    "y_top": 1,
    "y_bottom": 26,
    "y_center": 14,
    "raw_h": 25,
    "x_left": 2,
    "x_right": 48
  },
  {
    "text": "백산동물병원",
    "confidence": 0.983726441860199,
    "y_top": 0,
    "y_bottom": 39,
    "y_center": 20,
    "raw_h": 39,
    "x_left": 95,
    "x_right": 329
  },
  {
    "text": "백산동물명원",
    "confidence": 0.8984214663505554,
    "y_top": 0,
    "y_bottom": 20,
    "y_center": 10,
    "raw_h": 20,
    "x_left": 341,
    "x_right": 505
  },
  {
    "text": "전료기록-아전경(양의)",
    "confidence": 0.7902871966362,
    "y_top": 0,
    "y_bottom": 27,
    "y_center": 14,
    "raw_h": 27,
    "x_left": 1054,
    "x_right": 1353
  },
  {
    "text": "서울특별시 강남구 역삼동 725-15 Tel) 02-1644-5653",
    "confidence": 0.9538276791572571,
    "y_top": 7,
    "y_bottom": 49,
    "y_center": 28,
    "raw_h": 42,
    "x_left": 336,
    "x_right": 849
  },
  {
    "text": "Laboratory R

In [18]:
# 전체 요약
print('\n' + '#' * 80)
print('SUMMARY')
print('#' * 80)
for r in all_results:
    tests = len((r.get('doc_result') or {}).get('tests', []) or [])
    print(f"- {r['image']}: valid={r['valid']} tests={tests} elapsed={r['elapsed_s']:.2f}s")



################################################################################
SUMMARY
################################################################################
- 20241221_0.jpg: valid=True tests=23 elapsed=9.59s
- 20241221_1.jpg: valid=True tests=9 elapsed=6.00s


In [19]:
# (추가) 각 문서별 상세 출력 (헤더 + 표 형식 JSON)
print('\n' + '=' * 80)
print('📋 각 문서별 상세 결과')
print('=' * 80)

for idx, r in enumerate(all_results):
    doc = r.get('doc_result') or {}
    tests = doc.get('tests', []) or []
    
    print(f'\n🧾 Step {idx + 1}: {r["image"]}')
    print(f'⏱️ elapsed: {r["elapsed_s"]:.2f}s | tests: {len(tests)}')
    print()
    
    # 메타 정보 출력
    print(f'🏥 hospital_name    : {doc.get("hospital_name") or "(None)"}')
    print(f'👤 client_name      : {doc.get("client_name") or "(None)"}')
    print(f'🐾 patient_name     : {doc.get("patient_name") or "(None)"}')
    print(f'🗓  inspection_date : {doc.get("inspection_date") or "(None)"}')
    print()

    if tests:
        # 표 헤더
        print('code        | value | unit    | reference_min | reference_max')
        print('------------+-------+---------+---------------+--------------')
        
        # 각 테스트 항목
        for t in tests:
            code = (t.get('code') or 'UNKNOWN')[:12].ljust(12)
            value = str(t.get('value') or '')[:5].rjust(5)
            unit = (t.get('unit') or 'UNKNOWN')[:7].ljust(7)
            ref_min = str(t.get('reference_min') or 'UNKNOWN')[:13].rjust(13)
            ref_max = str(t.get('reference_max') or 'UNKNOWN')[:13].rjust(13)
            
            print(f'{code}| {value} | {unit} | {ref_min} | {ref_max}')
    else:
        print('(추출된 테스트 항목 없음)')



📋 각 문서별 상세 결과

🧾 Step 1: 20241221_0.jpg
⏱️ elapsed: 9.59s | tests: 23

🏥 hospital_name    : (None)
👤 client_name      : Patient: 양희
🐾 patient_name     : 양희
🗓  inspection_date : 2024-12-21

code        | value | unit    | reference_min | reference_max
------------+-------+---------+---------------+--------------
LYMPHO      |  17.1 | %       |       UNKNOWN |       UNKNOWN
MONO        |   6.3 | %       |       UNKNOWN |       UNKNOWN
EOS         |   3.7 | %       |       UNKNOWN |       UNKNOWN
BASO        |   0.4 | %       |       UNKNOWN |       UNKNOWN
NEUT        | 10.09 | K/µL    |          1.15 |         10.29
LYMPH       |  2.38 | K/µL    |          0.92 |          6.88
MONO        |  0.88 | K/µL    |          0.05 |          0.67
EOS         |  0.51 | K/µL    |          0.17 |          1.57
BASO        |  0.06 | K/µL    |          0.01 |          0.26
PLT         | 296.0 | K/µL    |         151.0 |         600.0
MPV         |  22.2 | fL      |       UNKNOWN |       UNKNOWN
PCT 

In [20]:
# (스마트 병합) LabReportExtractor.merge_extractions 사용
# - 동일 검사일/header_shape이면 병합
# - 별개 검사지면 분리 유지
# - 중복 code+unit 자동 제거

doc_results = [r['doc_result'] for r in all_results]
merge_envelope = lab_report_extractor.merge_extractions(doc_results)

print('\n' + '=' * 80)
print('🔀 SMART MERGE (LabReportExtractor.merge_extractions)')
print('=' * 80)
print(f'입력: {len(doc_results)}개 문서')
print(f'병합 후: {merge_envelope.meta.merged_len}개 문서')
print(f'빈 문서 제외: {merge_envelope.meta.pruned_empty}개')
print(f'중복 제거: {merge_envelope.meta.before_dedup} → {merge_envelope.meta.after_dedup} tests')
print()

for idx, merged_doc in enumerate(merge_envelope.data.merged):
    tests = merged_doc.get('tests', [])
    print(f'\n📄 병합 문서 #{idx + 1}')
    print(f'🏥 hospital_name    : {merged_doc.get("hospital_name") or "(None)"}')
    print(f'👤 client_name      : {merged_doc.get("client_name") or "(None)"}')
    print(f'🐾 patient_name     : {merged_doc.get("patient_name") or "(None)"}')
    print(f'🗓  inspection_date : {merged_doc.get("inspection_date") or "(None)"}')
    print(f'📊 tests count      : {len(tests)}')
    print()
    
    if tests:
        print('code        | value | unit    | reference_min | reference_max')
        print('------------+-------+---------+---------------+--------------')
        for t in tests:
            code = (t.get('code') or 'UNKNOWN')[:12].ljust(12)
            value = str(t.get('value') or '')[:5].rjust(5)
            unit = (t.get('unit') or 'UNKNOWN')[:7].ljust(7)
            ref_min = str(t.get('reference_min') or 'UNKNOWN')[:13].rjust(13)
            ref_max = str(t.get('reference_max') or 'UNKNOWN')[:13].rjust(13)
            print(f'{code}| {value} | {unit} | {ref_min} | {ref_max}')



🔀 SMART MERGE (LabReportExtractor.merge_extractions)
입력: 2개 문서
병합 후: 1개 문서
빈 문서 제외: 0개
중복 제거: 32 → 23 tests


📄 병합 문서 #1
🏥 hospital_name    : (None)
👤 client_name      : Patient: 양희
🐾 patient_name     : 양희
🗓  inspection_date : 2024-12-21
📊 tests count      : 23

code        | value | unit    | reference_min | reference_max
------------+-------+---------+---------------+--------------
LYMPHO      |  17.1 | %       |       UNKNOWN |       UNKNOWN
MONO        |   6.3 | %       |       UNKNOWN |       UNKNOWN
EOS         |   3.7 | %       |       UNKNOWN |       UNKNOWN
BASO        |   0.4 | %       |       UNKNOWN |       UNKNOWN
NEUT        | 10.09 | K/µL    |          1.15 |         10.29
LYMPH       |  2.38 | K/µL    |          0.92 |          6.88
MONO        |  0.88 | K/µL    |          0.05 |          0.67
EOS         |  0.51 | K/µL    |          0.17 |          1.57
BASO        |  0.06 | K/µL    |          0.01 |          0.26
PLT         | 296.0 | K/µL    |         151.0 |       